In [286]:
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp, CTransformers

import gradio as gr

from dotenv import load_dotenv
import os

In [287]:
class ChatPDF:

    def __init__(self, documents_folder: str, vectordb_folder: str, model_path: str, sentence_embedding_model: str, temperature: float = 0.1):
        """
        Constrói todos os atributos necessários para o objeto ChatPDF

        Args:
            documents_folder (str): Caminho/Pasta com os documentos que serão carregados para conversação no Chat
            vectordb_folder (str): Caminho/Pasta onde ficarão os arquivos do Chroma (Vector DB)
            model_path (str): Caminho/Pasta que aponta para o modelo LLM a ser utilizado
            sentence_embedding_model (str): Nome do modelo de Embedding que será usado para gerar os tokens dos documentos
            temperature (float, optional): Temperatura para calibrar o nível de aleatoriedade das respostas. O padrão é 0.1 (Muito determinístico, pouco aleatório)
        """
        self.documents_folder = documents_folder
        self.vectordb_folder = vectordb_folder
        self.model_path = model_path
        self.sentence_embedding_model = sentence_embedding_model
        self.temperature = temperature
        self.pages = []
        self.chunks = []

    def load(self) -> int:
        """
        Realiza a carga dos documentos do caminho/pasta definido no atributo documents_folder.

        Returns:
            int: Quantidade total de páginas carregadas de todos os arquivos PDF
        """

        loader = DirectoryLoader(
            self.documents_folder,
            glob="**/*.pdf",
            loader_cls=PyMuPDFLoader,
            show_progress=True,
            use_multithreading=True
        )

        self.pages = loader.load()

        return len(self.pages)
    
    def split(self, chunk_size: int = 800, chunk_overlap: int = 350) -> int:
        """
        Realiza o split das páginas em chunks para armazenar no Vector DB

        Args:
            chunk_size (int, optional): Quantidade máxima de caracteres de cada chunk. O padrão é 1500.
            chunk_overlap (int, optional): Quantidade de caracteres de overlap entre chunks. O padrão é 150.

        Returns:
            int: Quantidade total de chunks de todas as páginas de todos os documentos carregados
        """

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )

        self.chunks = text_splitter.split_documents(self.pages)

        return len(self.chunks)
    
    def get_embeddings(self):
        """
        Obtem os embeddings do modelo de linguagem definido no atributo sentence_embedding_model.
        """
        self.embeddings = SentenceTransformerEmbeddings(model_name=self.sentence_embedding_model)

    def store(self):
        """
        Armazena os chunks de todos os documentos no Vector DB, utilizando o embedding definido. 
        """
        vectordb = Chroma.from_documents(
            documents=self.chunks,
            embedding=self.embeddings,
            persist_directory=self.vectordb_folder
        )

        vectordb.persist()

        self.vectordb = vectordb

    def create_llm(self):
        """
        Cria uma LLM local, com base no modelo definido no atributo model_path.
        """
        
        self.llm = LlamaCpp(model_path=self.model_path, verbose=True, n_ctx=2048, temperature=self.temperature)

    def create_retriever(self):
        """
        Cria um retriever de documentos com base do Vector DB já carregado com os documentos.
        """
        self.retriever = self.vectordb.as_retriever(search_kwargs={'k':1})

    def create_qa_session(self):
        """
        Cria uma sessão de QA, usando o LLM e Retriever já instanciados.
        """

        PROMPT_TEMPLATE = """
        Você é um especialista em tradução, responda somente em Português. 
        Caso a pergunta seja fora do contexto, diga que o tema é fora do tópico.:
        {context}
        Responda a pergunta com base no contexto: {question}
        Answer: """

        QA_CHAIN_PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

        self.qa = RetrievalQA.from_chain_type(
            self.llm,
            'stuff',
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={'prompt': QA_CHAIN_PROMPT}
        )

In [288]:
load_dotenv()

MODEL_PATH = os.getenv('MODEL_PATH')
VECTORDB_FOLDER = os.getenv('VECTORDB_FOLDER')
DOCUMENTS_FOLDER = os.getenv('DOCUMENTS_FOLDER')
SENTENCE_EMBEDDING_MODEL = os.getenv('SENTENCE_EMBEDDING_MODEL')

In [289]:
chat = ChatPDF(DOCUMENTS_FOLDER, VECTORDB_FOLDER, MODEL_PATH, SENTENCE_EMBEDDING_MODEL, temperature=0.7)

In [290]:
chat.load()

100%|██████████| 1/1 [00:00<00:00, 140.12it/s]


13

In [291]:
chat.split()

13

In [292]:
chat.get_embeddings()

In [293]:
chat.store()

In [294]:
chat.create_llm()

llama_model_loader: loaded meta data with 21 key-value pairs and 364 tensors from /home/michaelalvesribeiro/Workspace/llama_Ollama3/models/sha256-ad5d403531ddb282b3a90d3bb6ea49a2e389e39fc659e9f9c79c58beb485bcae (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = falcon
llama_model_loader: - kv   1:                               general.name str              = Falcon
llama_model_loader: - kv   2:                      falcon.context_length u32              = 2048
llama_model_loader: - kv   3:                  falcon.tensor_data_layout str              = jploski
llama_model_loader: - kv   4:                    falcon.embedding_length u32              = 4096
llama_model_loader: - kv   5:                 falcon.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                         falcon.block_count u

In [295]:
# Cria o retriever, que irá recuperar os documentos do Vector DB, com base nos Prompts e usando a LLM
chat.create_retriever()

In [296]:
# Cria uma sessão de chat para Q&A, com base no LLM e Retriever
chat.create_qa_session()

In [297]:
# Front-End da Aplicação Web - Gradio

chat_history = []

with gr.Blocks() as demo:

    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    chat_history = []
    
    def user(user_message, chat_history):
        
        # Retorna resposta da LLM, através da sessão de Q&A
        result = chat.qa({"query": user_message})
        
        # Realiza um append na tela do chat, contendo a mensagem do usuário e a resposta do modelo
        chat_history.append((user_message, result["result"]))

        return gr.update(value=""), chat_history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.41.2, however version 4.29.0 is available, please upgrade.
--------



llama_print_timings:        load time =     852.84 ms
llama_print_timings:      sample time =     100.32 ms /   256 runs   (    0.39 ms per token,  2551.89 tokens per second)
llama_print_timings: prompt eval time =   12117.01 ms /   144 tokens (   84.15 ms per token,    11.88 tokens per second)
llama_print_timings:        eval time =   56362.35 ms /   256 runs   (  220.17 ms per token,     4.54 tokens per second)
llama_print_timings:       total time =   68786.07 ms /   400 tokens


Keyboard interruption in main thread... closing server.
